## Extended Algorithm Testing

In [4]:
import os
import matplotlib.pyplot as plt
from Compressor import GolombRiceCompressor
from InputFileGenerator import InputFileGenerator


def compression_ratio(compressor, infile_path):
    comp_file = infile_path + '.bin'
    # Compress and measure sizes
    compressor.compress_file(infile_path, comp_file)
    size_orig = os.path.getsize(infile_path)
    size_comp = os.path.getsize(comp_file)
    # Clean up compressed file
    os.remove(comp_file)
    return size_orig / size_comp


def count_values(file_path):
    with open(file_path, 'r') as f:
        tokens = f.read().split()
    return len(tokens)


def main():
    # Inicialización
    comp = GolombRiceCompressor()
    gen = InputFileGenerator(line_length=1000)

    # Archivos originales
    original_files = ['files/file11111.txt', 'files/file22222.txt']
    # Tomamos N del primer fichero para las simulaciones
    N = count_values(original_files[0])

    labels = []
    ratios = []

    # Probar ficheros originales
    for fpath in original_files:
        r = compression_ratio(comp, fpath)
        labels.append(os.path.basename(fpath))
        ratios.append(r)

    # Métodos de generación sintética
    methods = {
        'random_walk': lambda path: gen.generate_random_walk(path, N, sigma=1.0),
        'noise':       lambda path: gen.generate_noise(path, N, low=0, high=10000),
        'periodic':    lambda path: gen.generate_periodic(path, N, amplitude=2000, periods=5, offset=5000),
        'mixed':       lambda path: gen.generate_mixed(path, N),
    }

    # Para cada método, generar 5 ficheros, calcular ratio y promediar
    for name, generate in methods.items():
        rs = []
        for i in range(5):
            gen_path = f'files/{name}_{i}.txt'
            generate(gen_path)
            rs.append(compression_ratio(comp, gen_path))
            os.remove(gen_path)
        mean_r = sum(rs) / len(rs)
        labels.append(name)
        ratios.append(mean_r)

    # Plot de ratios
    plt.figure()
    plt.bar(labels, ratios)
    plt.ylabel('Compression Ratio')
    plt.xticks(rotation=45)
    plt.title('Compression Ratios: Original vs Synthetic')
    plt.tight_layout()
    # Guardar figura
    plt.savefig('compression_ratios.png')
    plt.show()


if __name__ == '__main__':
    main()


KeyboardInterrupt: 